# Combine angle and distance

In [ ]:
import numpy as np
import copy
import matplotlib.pyplot as plt
from scipy import optimize
from matplotlib.colors import ListedColormap

import psutil
import os
import gc

np.random.seed([170786])

In [ ]:
def read_files(route):
    file = open(f"{route}","r")
    info_r1 = file.readlines()
    file.close()

    info = psutil.virtual_memory()
    print(psutil.Process(os.getpid()).memory_info().rss)
    print(info.total)
    print(info.percent)
    print(psutil.cpu_count())



    info_s1 = []
    for i in range (len(info_r1)):
        info_s1.append(info_r1[i].split())
    info_r1 = []

    print("---------------------------------------------------------")
    info = psutil.virtual_memory()
    print(psutil.Process(os.getpid()).memory_info().rss)
    print(info.total)
    print(info.percent)
    print(psutil.cpu_count())
    
    Natom_s1 = int(info_s1[3][0])
    index_s1 = []
    for i in range (len(info_s1)):
        if info_s1[i] == ['ITEM:', 'TIMESTEP']:
            index_s1.append(i)

    splited_trj1 = []
    for j in range (len(index_s1)):
        frame = np.zeros((Natom_s1,5))
        for i in range (Natom_s1):
            frame[i,0] = int(info_s1[index_s1[j]+i+9][0])
            frame[i,1] = float(info_s1[index_s1[j]+i+9][1])
            frame[i,2] = float(info_s1[index_s1[j]+i+9][2])
            frame[i,3] = float(info_s1[index_s1[j]+i+9][3])
            frame[i,4] = float(info_s1[index_s1[j]+i+9][4])
        splited_trj1.append([frame,info_s1[index_s1[j]+5:index_s1[j]+8]])
    
    return splited_trj1

In [ ]:
def wrap_coordiante(pos1,bx,by,bz):
    new_pos = np.zeros((1,3))
    
    new_pos[0,0] = pos1[0][0]-bx*np.floor(pos1[0][0]/bx)
    new_pos[0,1] = pos1[0][1]-by*np.floor(pos1[0][1]/by)
    new_pos[0,2] = pos1[0][2]-bz*np.floor(pos1[0][2]/bz)

    return new_pos

In [ ]:
def wrap_distance(n1,n2,bx,by,bz):
    relevant_pos = np.zeros((1,3))
    new_coordiantes = np.zeros((1,3))
    
    relevant_pos[0,0] = n2[0,0]-n1[0,0]
    relevant_pos[0,1] = n2[0,1]-n1[0,1]
    relevant_pos[0,2] = n2[0,2]-n1[0,2]
    
    new_coordiantes[0,0] = relevant_pos[0,0]-bx*np.round(relevant_pos[0,0]/bx)
    new_coordiantes[0,1] = relevant_pos[0,1]-by*np.round(relevant_pos[0,1]/by)
    new_coordiantes[0,2] = relevant_pos[0,2]-bz*np.round(relevant_pos[0,2]/bz)
    
    return np.sqrt(new_coordiantes[0,0]**2+new_coordiantes[0,1]**2+new_coordiantes[0,2]**2)

In [ ]:
def mass_center2(pos1,pos2,low_x,low_y,low_z):
    c = np.zeros((1,3))
    
    c[0,0] = (pos1[0]+pos2[0])/2-low_x
    c[0,1] = (pos1[1]+pos2[1])/2-low_y
    c[0,2] = (pos1[2]+pos2[2])/2-low_z
    
    return c

In [ ]:
def center_distance(n1,n2,bx,by,bz):
    new_n1 = wrap_coordiante(n1,bx,by,bz)
    new_n2 = wrap_coordiante(n2,bx,by,bz)
    
    dis = wrap_distance(new_n1,new_n2,bx,by,bz)
    
    return dis

## Angle calculation

In [ ]:
def Normal_vector(pos1,pos2,pos3):
    vec1 = np.zeros((1,3))
    vec2 = np.zeros((1,3))

    vec1[0,0] = pos2[0]-pos1[0]
    vec1[0,1] = pos2[1]-pos1[1]
    vec1[0,2] = pos2[2]-pos1[2]

    vec2[0,0] = pos3[0]-pos1[0]
    vec2[0,1] = pos3[1]-pos1[1]
    vec2[0,2] = pos3[2]-pos1[2]

    # normal vector
    n1 = np.zeros((1,3))
    n1 = np.cross(vec1,vec2)
    
    n1 = n1/np.sqrt(n1[0,0]**2+n1[0,1]**2+n1[0,2]**2)

    return n1

In [ ]:
def included_anlge(n1,n2):
    
    included_angle = np.arccos(np.dot(n1,np.transpose(n2)))/np.pi*180

    return included_angle

In [ ]:
def wrap_angle(angle):
    
    if angle>=90 and angle<=180:
        t = 180-angle
    else:
        t = angle
        
    return t

## calculation

In [ ]:
def do1(target,traj,boundary_x,boundary_y,boundary_z,low_x,low_y,low_z):
        ## distribution
    N_index = []
    for i in range (len(traj[target][0])):
        if traj[target][0][i][1] == 14.007:
            N_index.append(i)

    a_distribution = []
    d_distribution = []
    a_masscenter = []
    d_masscenter = []
    for i in range (int(len(N_index)/2)):
        a_vector = Normal_vector(traj[target][0][N_index[2*i+1]-3][2:],traj[target][0][N_index[2*i+1]-1][2:],traj[target][0][N_index[2*i+1]+17][2:])
        d_vector = Normal_vector(traj[target][0][N_index[2*i+1]][2:],traj[target][0][N_index[2*i+1]+6][2:],traj[target][0][N_index[2*i+1]+10][2:])
        a_distribution.append(a_vector)
        d_distribution.append(d_vector)

        a_mass = mass_center2(traj[target][0][N_index[2*i+1]-4][2:],traj[target][0][N_index[2*i+1]-1][2:],low_x,low_y,low_z)
        d_mass = mass_center2(traj[target][0][N_index[2*i+1]][2:],traj[target][0][N_index[2*i+1]+7][2:],low_x,low_y,low_z)
        a_masscenter.append(a_mass)
        d_masscenter.append(d_mass)
    
    return  a_distribution,d_distribution,a_masscenter,d_masscenter

In [ ]:
def scan_self(a_angle,a_mass,boundary_x,boundary_y,boundary_z):
    
    information = []
    for i in range (len(a_mass)):
        for j in range (i+1,len(a_mass)):
            dis = center_distance(a_mass[i],a_mass[j],boundary_x,boundary_y,boundary_z)
            if dis<dis_limit:
                ang = included_anlge(a_angle[i],a_angle[j])
                information.append([dis,ang[0][0]])
                
    return information

In [ ]:
def scan_others(a_angle,a_mass,d_angle,d_mass,boundary_x,boundary_y,boundary_z):
    
    information = []
    for i in range (len(a_mass)):
        for j in range (len(d_mass)):
            dis = center_distance(a_mass[i],d_mass[j],boundary_x,boundary_y,boundary_z)
            if dis<dis_limit:
                ang = included_anlge(a_angle[i],d_angle[j])
                information.append([dis,ang[0][0]])
                
    return information

In [ ]:
def scan_exclude(a_angle,a_mass,d_angle,d_mass,boundary_x,boundary_y,boundary_z):
    
    information = []
    for i in range (len(a_mass)):
        for j in range (len(d_mass)):
            if i!=j:
                dis = center_distance(a_mass[i],d_mass[j],boundary_x,boundary_y,boundary_z)
                if dis<dis_limit:
                    ang = included_anlge(a_angle[i],d_angle[j])
                    information.append([dis,ang[0][0]])
                
    return information

# Calculate

In [ ]:
def get_map(traj,target,type_):
    
    low_x = float(traj[target][1][0][0])
    low_y = float(traj[target][1][1][0])
    low_z = float(traj[target][1][2][0])
    
    boundary_x = float(traj[target][1][0][1])-low_x
    boundary_y = float(traj[target][1][1][1])-low_y
    boundary_z = float(traj[target][1][2][1])-low_z
    
    a_distribution,d_distribution,a_masscenter,d_masscenter = do1(target,traj,boundary_x,boundary_y,boundary_z,low_x,low_y,low_z)

    n_dis_bins = int(dis_limit/dis_interval)
    n_ang_bins = int(90/angle_interval)
    data_plot = np.zeros((n_dis_bins,n_ang_bins))
    
    if type_ == "AA":
        information_list = scan_self(a_distribution,a_masscenter,boundary_x,boundary_y,boundary_z)
    elif type_ == "DA":
        information_list = scan_others(a_distribution,a_masscenter,d_distribution,d_masscenter,boundary_x,boundary_y,boundary_z)
    elif type_ =="DA_exclude":
        information_list = scan_exclude(a_distribution,a_masscenter,d_distribution,d_masscenter,boundary_x,boundary_y,boundary_z)
    elif type_ == "DD":
        information_list = scan_self(d_distribution,d_masscenter,boundary_x,boundary_y,boundary_z)
    
    for id_ in range (len(information_list)):
        dis_loc = int(np.floor(information_list[id_][0]/dis_interval))
        ang_loc = int(np.floor(wrap_angle(information_list[id_][1])/angle_interval))
        data_plot[dis_loc,ang_loc] += 1/(4/3*np.pi*(((dis_loc+1)*dis_interval)**3-(dis_loc*dis_interval)**3))
        
    return data_plot

In [ ]:
def plot_map_ori(data_plot_all,id_,type_):
    # discrete color scheme
    # cMap = ListedColormap(['white', 'green', 'blue','red'])
    cm = plt.cm.get_cmap("Blues")
    
    data_plot = np.zeros((np.shape(data_plot_all[0])[0],np.shape(data_plot_all[0])[1]))
    for i in range (np.shape(data_plot_all[0])[0]):
        for j in range (np.shape(data_plot_all[0])[1]):
            for t in range (len(data_plot_all)):
                data_plot[i,j] += data_plot_all[t][i,j]
            data_plot[i,j] /= len(data_plot_all)

## For single frame
#     data_plot = np.zeros((np.shape(data_plot_all)[0],np.shape(data_plot_all)[1]))        
#     for i in range (np.shape(data_plot_all)[0]):
#         for j in range (np.shape(data_plot_all)[1]):
#             data_plot[i,j] += data_plot_all[i,j]

            
    # data
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111)
#     heatmap = ax.pcolor(np.transpose(data_plot/np.max(data_plot)), cmap=cm)
    heatmap = ax.pcolor(np.transpose(data_plot), cmap=cm)
    

    # legend
    cbar = plt.colorbar(heatmap)
#     cbar.set_label('$P (\Theta_{AA},r_{AA})$', rotation=90,size=15)

    # put the major ticks at the middle of each cell
    ax.set_xticks(np.arange(data_plot.shape[0]/10+1)*10, minor=False)
    ax.set_yticks(np.arange(data_plot.shape[1]/4.5)*5, minor=False)

    # labels
    column_labels = [0,5,10,15,20,25]
    row_labels = [0,10,20,30,40,50,60,70,80,90]
    ax.set_xticklabels(column_labels, minor=False)
    ax.set_yticklabels(row_labels, minor=False)
#     plt.xlabel("$\ r_{AA}$ $ (\AA)$",size=15)
#     plt.ylabel("$\Theta_{AA}$  $(^\circ)$",size=15)
    plt.title(f"{name} {id_}%",size=20)

    
    if type_ == "AA":
        cbar.set_label('$P (\Theta_{AA},r_{AA})$', rotation=90,size=15)
        plt.xlabel("$\ r_{AA}$ $ (\AA)$",size=15)
        plt.ylabel("$\Theta_{AA}$  $(^\circ)$",size=15)
    elif type_ == "DD":
        cbar.set_label('$P (\Theta_{DD},r_{DD})$', rotation=90,size=15)
        plt.xlabel("$\ r_{DD}$ $ (\AA)$",size=15)
        plt.ylabel("$\Theta_{DD}$  $(^\circ)$",size=15)
    else:
        cbar.set_label('$P (\Theta_{DA},r_{DA})$', rotation=90,size=15)
        plt.xlabel("$\ r_{DA}$ $ (\AA)$",size=15)
        plt.ylabel("$\Theta_{DA}$  $(^\circ)$",size=15)

    plt.savefig(f"21Dec-rerun/{temperature}_{name}_{type_}_{id_}%.pdf")
    
    plt.show()

In [ ]:
def write_data(data_plot_all,name,strain,type_):
    
    data_plot = np.zeros((np.shape(data_plot_all[0])[0],np.shape(data_plot_all[0])[1]))
    for i in range (np.shape(data_plot_all[0])[0]): # distance
        for j in range (np.shape(data_plot_all[0])[1]): # angle
            for t in range (len(data_plot_all)):
                data_plot[i,j] += data_plot_all[t][i,j]
            data_plot[i,j] /= len(data_plot_all)
    
    file = open(f"21Dec-rerun/{name}_{strain}_{type_}.txt","w")
    
    file.write("distance from 0 - 25 (interval 0.5, unit Angstrom), angle from 0 - 90 (interval 2, unit degree).")
    file.write("\n")
    file.write("every line has the same distance information, and increasing angle. Line starts from short distance.")
    file.write("\n")
    
    file.write("\n")
    for i in range (np.shape(data_plot_all[0])[0]):
        for j in range (np.shape(data_plot_all[0])[1]):
            file.write(f"{data_plot[i,j]}"+" ")
        file.write("\n")
    
    file.close()

## M
### 0%

In [ ]:
name = "PDKCM"
temperature = 1.2

dis_limit = 25
dis_interval = 0.5
angle_interval = 2

splited_trj1 = read_files("m/s0-ta.trj")

In [ ]:
data_m_0_DD = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_m_0_DD.append(get_map(splited_trj1,i*10,"DD"))

plot_map_ori(data_m_0_DD,0,"DD")

In [ ]:
data_m_0_AA = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_m_0_AA.append(get_map(splited_trj1,i*10,"AA"))
plot_map_ori(data_m_0_AA,0,"AA")

In [ ]:
data_m_0_AD_exclude = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_m_0_AD_exclude.append(get_map(splited_trj1,i*10,"DA_exclude"))
plot_map_ori(data_m_0_AD_exclude,0,"DA_exclude")

In [ ]:
write_data(data_m_0_DD,"PDKCM",0,"DD")
write_data(data_m_0_AA,"PDKCM",0,"AA")
write_data(data_m_0_AD_exclude,"PDKCM",0,"AD_ex")

## 50%

In [ ]:
name = "PDKCM"
temperature = 1.2

dis_limit = 25
dis_interval = 0.5
angle_interval = 2

splited_trj1 = read_files("m/s10_ta.trj")

In [ ]:
data_m_50_DD = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_m_50_DD.append(get_map(splited_trj1,i*10,"DD"))

plot_map_ori(data_m_50_DD,50,"DD")

In [ ]:
data_m_50_AA = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_m_50_AA.append(get_map(splited_trj1,i*10,"AA"))
plot_map_ori(data_m_50_AA,50,"AA")

In [ ]:
data_m_50_AD_exclude = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_m_50_AD_exclude.append(get_map(splited_trj1,i*10,"DA_exclude"))
plot_map_ori(data_m_50_AD_exclude,50,"DA_exclude")

In [ ]:
write_data(data_m_50_DD,"PDKCM",50,"DD")
write_data(data_m_50_AA,"PDKCM",50,"AA")
write_data(data_m_50_AD_exclude,"PDKCM",50,"AD_ex")

## 100%

In [ ]:
name = "PDKCM"
temperature = 1.2

dis_limit = 25
dis_interval = 0.5
angle_interval = 2

splited_trj1 = read_files("m/s20_ta.trj")

In [ ]:
data_m_100_DD = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_m_100_DD.append(get_map(splited_trj1,i*10,"DD"))
plot_map_ori(data_m_100_DD,100,"DD")

In [ ]:
data_m_100_AA = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_m_100_AA.append(get_map(splited_trj1,i*10,"AA"))
plot_map_ori(data_m_100_AA,100,"AA")

In [ ]:
data_m_100_AD_exclude = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_m_100_AD_exclude.append(get_map(splited_trj1,i*10,"DA_exclude"))
plot_map_ori(data_m_100_AD_exclude,100,"DA_exclude")

In [ ]:
write_data(data_m_100_DD,"PDKCM",100,"DD")
write_data(data_m_100_AA,"PDKCM",100,"AA")
write_data(data_m_100_AD_exclude,"PDKCM",100,"AD_ex")

# P

### 0%

In [ ]:
name = "PDKCP"
sname = "p"
temperature = 1.2

dis_limit = 25
dis_interval = 0.5
angle_interval = 2


splited_trj1 = read_files(f"{sname}/s0_ta.trj")

In [ ]:
data_p_0_DD = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_p_0_DD.append(get_map(splited_trj1,i*10,"DD"))
    
plot_map_ori(data_p_0_DD,0,"DD")

In [ ]:
data_p_0_AA = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_p_0_AA.append(get_map(splited_trj1,i*10,"AA"))
    
plot_map_ori(data_p_0_AA,0,"AA")

In [ ]:
data_p_0_AD_exclude = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_p_0_AD_exclude.append(get_map(splited_trj1,i*10,"DA_exclude"))
    
plot_map_ori(data_p_0_AD_exclude,0,"DA_exclude")

In [ ]:
write_data(data_p_0_DD,"PDKCP",0,"DD")
write_data(data_p_0_AA,"PDKCP",0,"AA")
write_data(data_p_0_AD_exclude,"PDKCP",0,"AD_ex")

### 50%

In [ ]:
name = "PDKCP"
sname = "p"
temperature = 1.2

dis_limit = 25
dis_interval = 0.5
angle_interval = 2

splited_trj1 = read_files(f"{sname}/s10_ta.trj")

In [ ]:
data_p_50_DD = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_p_50_DD.append(get_map(splited_trj1,i*10,"DD"))
plot_map_ori(data_p_50_DD,50,"DD")

In [ ]:
data_p_50_AA = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_p_50_AA.append(get_map(splited_trj1,i*10,"AA"))
plot_map_ori(data_p_50_AA,50,"AA")

In [ ]:
data_p_50_AD_exclude = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_p_50_AD_exclude.append(get_map(splited_trj1,i*10,"DA_exclude"))
plot_map_ori(data_p_50_AD_exclude,50,"DA_exclude")

In [ ]:
write_data(data_p_50_DD,"PDKCP",50,"DD")
write_data(data_p_50_AA,"PDKCP",50,"AA")
write_data(data_p_50_AD_exclude,"PDKCP",50,"AD_ex")

### 100%

In [ ]:
name = "PDKCP"
sname = "p"
temperature = 1.2

dis_limit = 25
dis_interval = 0.5
angle_interval = 2


splited_trj1 = read_files(f"{sname}/s20_ta.trj")

In [ ]:
data_p_100_DD = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_p_100_DD.append(get_map(splited_trj1,i*10,"DD"))
plot_map_ori(data_p_100_DD,100,"DD")

In [ ]:
data_p_100_AA = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_p_100_AA.append(get_map(splited_trj1,i*10,"AA"))
plot_map_ori(data_p_100_AA,100,"AA")

In [ ]:
data_p_100_AD_exclude = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_p_100_AD_exclude.append(get_map(splited_trj1,i*10,"DA_exclude"))
plot_map_ori(data_p_100_AD_exclude,100,"DA_exclude")

In [ ]:
write_data(data_p_100_DD,"PDKCP",100,"DD")
write_data(data_p_100_AA,"PDKCP",100,"AA")
write_data(data_p_100_AD_exclude,"PDKCP",100,"AD_ex")

# H

### 0%

In [ ]:
name = "PDKCH"
sname = "h"
temperature = 1.2

dis_limit = 25
dis_interval = 0.5
angle_interval = 2

splited_trj1 = read_files(f"{sname}/s0_ta.trj")
splited_trj2 = read_files(f"{sname}/s0_ta-2.trj")

In [ ]:
data_h_0_DD = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_h_0_DD.append(get_map(splited_trj1,10*i,"DD"))
    
for i in range (int(np.floor(len(splited_trj2)/10))+1):
    data_h_0_DD.append(get_map(splited_trj2,10*i,"DD"))
    
plot_map_ori(data_h_0_DD,0,"DD")

In [ ]:
data_h_0_AA = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_h_0_AA.append(get_map(splited_trj1,10*i,"AA"))
    
for i in range (int(np.floor(len(splited_trj2)/10))+1):
    data_h_0_AA.append(get_map(splited_trj2,10*i,"AA"))
    
plot_map_ori(data_h_0_AA,0,"AA")

In [ ]:
data_h_0_AD_exclude = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_h_0_AD_exclude.append(get_map(splited_trj1,i*10,"DA_exclude"))
    
for i in range (int(np.floor(len(splited_trj2)/10))+1):
    data_h_0_AD_exclude.append(get_map(splited_trj2,i*10,"DA_exclude"))

plot_map_ori(data_h_0_AD_exclude,0,"DA_exclude")

In [ ]:
write_data(data_h_0_DD,"PDKCH",0,"DD")
write_data(data_h_0_AA,"PDKCH",0,"AA")
write_data(data_h_0_AD_exclude,"PDKCH",0,"AD_ex")

### 50%

In [ ]:
name = "PDKCH"
sname = "h"
temperature = 1.2

dis_limit = 25
dis_interval = 0.5
angle_interval = 2


splited_trj1 = read_files(f"{sname}/s10_ta.trj")
splited_trj2 = read_files(f"{sname}/s10_ta-2.trj")

In [ ]:
data_h_50_DD = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_h_50_DD.append(get_map(splited_trj1,10*i,"DD"))
    
for i in range (int(np.floor(len(splited_trj2)/10))+1):
    data_h_50_DD.append(get_map(splited_trj2,10*i,"DD"))
    
plot_map_ori(data_h_50_DD,50,"DD")

In [ ]:
data_h_50_AA = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_h_50_AA.append(get_map(splited_trj1,10*i,"AA"))
    
for i in range (int(np.floor(len(splited_trj2)/10))+1):
    data_h_50_AA.append(get_map(splited_trj2,10*i,"AA"))
    
plot_map_ori(data_h_50_AA,50,"AA")

In [ ]:
data_h_50_AD_exclude = []
for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_h_50_AD_exclude.append(get_map(splited_trj1,i*10,"DA_exclude"))
    
for i in range (int(np.floor(len(splited_trj2)/10))+1):
    data_h_50_AD_exclude.append(get_map(splited_trj2,i*10,"DA_exclude"))

plot_map_ori(data_h_50_AD_exclude,50,"DA_exclude")

In [ ]:
write_data(data_h_50_DD,"PDKCH",50,"DD")
write_data(data_h_50_AA,"PDKCH",50,"AA")
write_data(data_h_50_AD_exclude,"PDKCH",50,"AD_ex")

### 100%

In [ ]:
name = "PDKCH"
sname = "h"
temperature = 1.2

dis_limit = 25
dis_interval = 0.5
angle_interval = 2

splited_trj1 = read_files(f"{sname}/s20_ta.trj")

In [ ]:
data_h_100_DD = []

for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_h_100_DD.append(get_map(splited_trj1,10*i,"DD"))

plot_map_ori(data_h_100_DD,100,"DD")

In [ ]:
data_h_100_AA = []

for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_h_100_AA.append(get_map(splited_trj1,10*i,"AA"))

plot_map_ori(data_h_100_AA,100,"AA")

In [ ]:
data_h_100_AD_exclude = []

for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_h_100_AD_exclude.append(get_map(splited_trj1,i*10,"DA_exclude"))

plot_map_ori(data_h_100_AD_exclude,100,"DA_exclude")

In [ ]:
write_data(data_h_100_DD,"PDKCH",100,"DD")
write_data(data_h_100_AA,"PDKCH",100,"AA")
write_data(data_h_100_AD_exclude,"PDKCH",100,"AD_ex")

# D

### 0%

In [ ]:
name = "PDKCD"
sname = "d"
temperature = 1.2

dis_limit = 25
dis_interval = 0.5
angle_interval = 2

splited_trj1 = read_files(f"{sname}/s0_ta.trj")
splited_trj2 = read_files(f"{sname}/s0_ta-2.trj")

In [ ]:
data_d_0_DD = []

for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_d_0_DD.append(get_map(splited_trj1,i*10,"DD"))
    
for i in range (int(np.floor(len(splited_trj2)/10))+1):
    data_d_0_DD.append(get_map(splited_trj2,10*i,"DD"))
    
plot_map_ori(data_d_0_DD,0,"DD")

In [ ]:
data_d_0_AA = []

for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_d_0_AA.append(get_map(splited_trj1,i*10,"AA"))
    
for i in range (int(np.floor(len(splited_trj2)/10))+1):
    data_d_0_AA.append(get_map(splited_trj2,10*i,"AA"))
    
plot_map_ori(data_d_0_AA,0,"AA")

In [ ]:
data_d_0_AD_exclude = []

for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_d_0_AD_exclude.append(get_map(splited_trj1,i*10,"DA_exclude"))
    
for i in range (int(np.floor(len(splited_trj2)/10))+1):
    data_d_0_AD_exclude.append(get_map(splited_trj2,10*i,"DA_exclude"))
    
plot_map_ori(data_d_0_AD_exclude,0,"DA_exclude")

In [ ]:
write_data(data_d_0_DD,"PDKCD",0,"DD")
write_data(data_d_0_AA,"PDKCD",0,"AA")
write_data(data_d_0_AD_exclude,"PDKCD",0,"AD_ex")

## 50%

In [ ]:
name = "PDKCD"
sname = "d"
temperature = 1.2

dis_limit = 25
dis_interval = 0.5
angle_interval = 2

splited_trj1 = read_files(f"{sname}/s10_ta.trj")

In [ ]:
data_d_50_DD = []

for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_d_50_DD.append(get_map(splited_trj1,i*10,"DD"))

plot_map_ori(data_d_50_DD,50,"DD")

In [ ]:
data_d_50_AA = []

for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_d_50_AA.append(get_map(splited_trj1,i*10,"AA"))
    
plot_map_ori(data_d_50_AA,50,"AA")

In [ ]:
data_d_50_AD_exclude = []

for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_d_50_AD_exclude.append(get_map(splited_trj1,i*10,"DA_exclude"))
    
plot_map_ori(data_d_50_AD_exclude,50,"DA_exclude")

In [ ]:
write_data(data_d_50_DD,"PDKCD",50,"DD")
write_data(data_d_50_AA,"PDKCD",50,"AA")
write_data(data_d_50_AD_exclude,"PDKCD",50,"AD_ex")

## 100%

In [ ]:
name = "PDKCD"
sname = "d"
temperature = 1.2

dis_limit = 25
dis_interval = 0.5
angle_interval = 2

splited_trj1 = read_files(f"{sname}/s20_ta.trj")

In [ ]:
data_d_100_DD = []

for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_d_100_DD.append(get_map(splited_trj1,i*10,"DD"))
    
plot_map_ori(data_d_100_DD,100,"DD")

In [ ]:
data_d_100_AA = []

for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_d_100_AA.append(get_map(splited_trj1,i*10,"AA"))
    
plot_map_ori(data_d_100_AA,100,"AA")

In [ ]:
data_d_100_AD_exclude = []

for i in range (int(np.floor(len(splited_trj1)/10))+1):
    data_d_100_AD_exclude.append(get_map(splited_trj1,i*10,"DA_exclude"))
    
plot_map_ori(data_d_100_AD_exclude,100,"DA_exclude")

In [ ]:
write_data(data_d_100_DD,"PDKCD",100,"DD")
write_data(data_d_100_AA,"PDKCD",100,"AA")
write_data(data_d_100_AD_exclude,"PDKCD",100,"AD_ex")